<a href="https://colab.research.google.com/github/Minh-A/Text_Mining/blob/main/log%20analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 라이브러리 불러오기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# **2. 데이터 불러오기 및 간단 EDA**

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/로그분석/data/train.csv')
vali = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/로그분석/data/validation_sample.csv')
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/로그분석/data/test.csv')
submission=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/로그분석/data/sample_submission.csv')

In [ ]:
print(train.shape, test.shape, vali.shape, submission.shape)

(472972, 3) (1418916, 2) (3, 1) (1418916, 2)


In [ ]:
train.head()

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...


In [ ]:
#test는 level값이 없습니다
test.head()

,id,full_log
0,1000000,"Feb 8 15:47:26 localhost kibana: {""type"":""err..."
1,1000001,"Sep 24 03:46:39 localhost kibana: {""type"":""err..."
2,1000002,type=SYSCALL msg=audit(1611888200.428:210563):...
3,1000003,"Jan 18 11:24:06 localhost kibana: {""type"":""err..."
4,1000004,type=SYSCALL msg=audit(1603081202.050:46851): ...


In [ ]:
train_df = train
test_df = test

In [ ]:
first_strings = pd.merge(train_df.loc[train_df.level!=0].full_log.apply(lambda x: x.split(' ')[0]).value_counts().rename('lv!=0'),
                         train_df.loc[train_df.level==0].full_log.apply(lambda x: x.split(' ')[0]).value_counts().rename('lv==0'),
                         how='outer',left_index=True,right_index=True).fillna(0).astype(int)

first_strings = first_strings.sort_values('lv==0',ascending=False)

In [ ]:
first_strings

,lv!=0,lv==0
Jan,6451,200574
Feb,2430,33114
Dec,1512,24370
Sep,801,22392
Oct,2910,20973
Nov,2895,18980
Mar,806,11368
level,812,1519
error:,0,459
E:,0,297


In [ ]:
train['full_log'].value_counts()

error: cannot open Packages database in                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               138
error: db5 error(-30973) from 

In [ ]:
train['level'].value_counts()

0    334065
1    132517
3      4141
5      2219
2        12
4        10
6         8
Name: level, dtype: int64

In [ ]:
# text 전처리로 문자만 남겨 보았다.
train['full_log'] = train['full_log'].str.replace(pat=r'[^A-Za-z]', repl=r'', regex=True)
test['full_log'] = test['full_log'].str.replace(pat=r'[^A-Za-z]', repl=r'', regex=True)

In [ ]:
#full_log에서 숫자는 마스킹 처리
train['full_log']=train['full_log'].str.replace(r'[0-9]', ' ')
test['full_log']=test['full_log'].str.replace(r'[0-9]', ' ')

In [ ]:
train['full_log'].value_counts()

In [ ]:
train['full_log'].value_counts()

Jan      :  :   localhost kibana: {"type":"error","@timestamp":"    -  -  T  :  :  Z","tags":["warning","stats-collection"],"pid":    ,"level":"error","error":{"message":"No Living connections","name":"Error","stack":"Error: No Living connections\n    at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:   :  )\n    at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:   : )\n    at process._tickCallback (internal/process/next_tick.js:  :  )"},"message":"No Living connections"}                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
#train['full_log'] => train_text로 list
#train['level']=> train_level로 array
train_text=list(train['full_log'])
train_level=np.array(train['level'])

In [ ]:
text_train, text_val, label_train, label_val = train_test_split(train['full_log'], train['level'], test_size=0.2, random_state=42, stratify=train['level'])

In [ ]:
import tensorflow as tf

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(text_train)
top_k = len(tokenizer.word_index)

x_train = tokenizer.texts_to_sequences(text_train)
x_val = tokenizer.texts_to_sequences(text_val)

max_length=100

x_train_vector = tf.keras.preprocessing.sequence.pad_sequences(
    x_train, maxlen=max_length, padding='post'
)

x_val_vector = tf.keras.preprocessing.sequence.pad_sequences(
    x_val, maxlen=max_length, padding='post'
)

In [ ]:
x_train_vector.shape

(378377, 100)

In [ ]:
x_val_vector.shape

(94595, 100)

In [ ]:
x_train_vector

array([[162,   8,  10, ...,   0,   0,   0],
       [ 42,   8,  24, ...,   0,   0,   0],
       [ 42,   8,  24, ...,   0,   0,   0],
       ...,
       [ 42,   8,  10, ...,   0,   0,   0],
       [ 42,   8,  10, ...,   0,   0,   0],
       [ 42,   8,  10, ...,   0,   0,   0]], dtype=int32)

In [ ]:
# # 검증 값
from sklearn import metrics

# def macro_f1(answer_df, submission_df):

#     submission_df = submission_df[submission_df['id'].isin(answer_df['id'])]

#     submission_df.index = range(submission_df.shape[0])

    

#     true = answer_df['level']

#     pred = submission_df['level']

    

#     score = metrics.f1_score(y_true=true, y_pred=pred, average='macro')

    

#     return score

In [ ]:
def macro_f1(answer_df, submission_df):    
    true = answer_df
    pred = submission_df

    score = metrics.f1_score(y_true=true, y_pred=pred, average='macro')
    return score    

In [ ]:
pred

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
label_val

91835     0
160352    0
194710    0
118233    0
207022    0
         ..
365308    0
372398    0
267991    0
360722    1
292629    0
Name: level, Length: 94595, dtype: int64

In [ ]:
macro_f1(label_val,pred)

0.8235142934209632

# **3. 데이터 전처리**

In [ ]:
#full_log에서 숫자는 마스킹 처리
train['full_log']=train['full_log'].str.replace(r'[0-9]', '<num>')
test['full_log']=test['full_log'].str.replace(r'[0-9]', '<num>')

In [ ]:
#train['full_log'] => train_text로 list
#train['level']=> train_level로 array
train_text=list(train['full_log'])
train_level=np.array(train['level'])

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
text_train, text_val, label_train, label_val = train_test_split(train['full_log'], train['level'], test_size=0.2, random_state=42)

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(text_train)
top_k = len(tokenizer.word_index)

x_train = tokenizer.texts_to_sequences(text_train)
x_val = tokenizer.texts_to_sequences(text_val)

max_length=100

x_train_vector = tf.keras.preprocessing.sequence.pad_sequences(
    x_train, maxlen=max_length, padding='post'
)

x_val_vector = tf.keras.preprocessing.sequence.pad_sequences(
    x_val, maxlen=max_length, padding='post'
)

In [ ]:
x_train_vector.shape

(378377, 100)

In [ ]:
x_val_vector.shape

(94595, 100)

In [ ]:
x_train_vector

array([[161,   1,   1, ...,   0,   0,   0],
       [  3,  53,   7, ...,   1, 332, 327],
       [ 43,   1,   1, ...,   0,   0,   0],
       ...,
       [  1,   1,   1, ...,   1,   1,   1],
       [  1,  56,   1, ...,  41,  51,  52],
       [ 43,   1,   1, ...,   0,   0,   0]], dtype=int32)

In [ ]:
#랜덤포레스트로 모델링
from sklearn.ensemble import RandomForestClassifier

forest=RandomForestClassifier(n_estimators=1000, random_state=42)

forest.fit(x_train_vector, label_train)

* https://injo.tistory.com/44

In [ ]:
# LGBM 모델
lgb = LGBMClassifier(n_estimators=400, random_state=42)
evals = [(x_val_vector, label_val)]
lgb.fit(x_train_vector, label_train, early_stopping_rounds=50, eval_metric='error', eval_set=evals, verbose=True)

[1]	valid_0's multi_logloss: 0.52187	valid_0's multi_error: 0.281865
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_logloss: 0.446025	valid_0's multi_error: 0.281791
[3]	valid_0's multi_logloss: 0.400969	valid_0's multi_error: 0.281907
[4]	valid_0's multi_logloss: 0.343779	valid_0's multi_error: 0.00288599
[5]	valid_0's multi_logloss: 0.308006	valid_0's multi_error: 0.00294942
[6]	valid_0's multi_logloss: 0.27198	valid_0's multi_error: 0.00272742
[7]	valid_0's multi_logloss: 0.255007	valid_0's multi_error: 0.00302342
[8]	valid_0's multi_logloss: 0.213276	valid_0's multi_error: 0.00251599
[9]	valid_0's multi_logloss: 0.205743	valid_0's multi_error: 0.00281199
[10]	valid_0's multi_logloss: 0.180026	valid_0's multi_error: 0.0026217
[11]	valid_0's multi_logloss: 0.174821	valid_0's multi_error: 0.00297056
[12]	valid_0's multi_logloss: 0.145301	valid_0's multi_error: 0.00253713
[13]	valid_0's multi_logloss: 0.148162	valid_0's multi_error: 0.00295999
[14]	va

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=400, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
# LGBM 모델
import xgboost as xgb ## XGBoost 불러오기
lgb = xgb.XGBClassifier(n_estimators=1000, random_state=42)
evals = [(x_val_vector, label_val)]
lgb.fit(x_train_vector, label_train, early_stopping_rounds=50, eval_metric='merror', eval_set=evals, verbose=True)

[0]	validation_0-merror:0.010349
Will train until validation_0-merror hasn't improved in 50 rounds.
[1]	validation_0-merror:0.010085
[2]	validation_0-merror:0.009884
[3]	validation_0-merror:0.009863
[4]	validation_0-merror:0.009631
[5]	validation_0-merror:0.009514
[6]	validation_0-merror:0.009356
[7]	validation_0-merror:0.009324
[8]	validation_0-merror:0.00925
[9]	validation_0-merror:0.009779
[10]	validation_0-merror:0.009525
[11]	validation_0-merror:0.00944
[12]	validation_0-merror:0.009113
[13]	validation_0-merror:0.009113
[14]	validation_0-merror:0.009123
[15]	validation_0-merror:0.0087
[16]	validation_0-merror:0.008711
[17]	validation_0-merror:0.008679
[18]	validation_0-merror:0.008425
[19]	validation_0-merror:0.007823
[20]	validation_0-merror:0.007844
[21]	validation_0-merror:0.007189
[22]	validation_0-merror:0.007115
[23]	validation_0-merror:0.007125
[24]	validation_0-merror:0.006977
[25]	validation_0-merror:0.006945
[26]	validation_0-merror:0.006945
[27]	validation_0-merror:0.00

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

* XGBClassifier
              (base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

* 0.9472021192302403

In [ ]:
# LGBM 모델
import xgboost as xgb ## XGBoost 불러오기
lgb = xgb.XGBClassifier(n_estimators=1500, random_state=42, n_jobs=-1)
evals = [(x_val_vector, label_val)]
lgb.fit(x_train_vector, label_train, early_stopping_rounds=50, eval_metric='merror', eval_set=evals, verbose=True)

[0]	validation_0-merror:0.010349
Will train until validation_0-merror hasn't improved in 50 rounds.
[1]	validation_0-merror:0.010085
[2]	validation_0-merror:0.009884
[3]	validation_0-merror:0.009863
[4]	validation_0-merror:0.009631
[5]	validation_0-merror:0.009514
[6]	validation_0-merror:0.009356
[7]	validation_0-merror:0.009324
[8]	validation_0-merror:0.00925
[9]	validation_0-merror:0.009779
[10]	validation_0-merror:0.009525
[11]	validation_0-merror:0.00944
[12]	validation_0-merror:0.009113
[13]	validation_0-merror:0.009113
[14]	validation_0-merror:0.009123
[15]	validation_0-merror:0.0087
[16]	validation_0-merror:0.008711
[17]	validation_0-merror:0.008679
[18]	validation_0-merror:0.008425
[19]	validation_0-merror:0.007823
[20]	validation_0-merror:0.007844
[21]	validation_0-merror:0.007189
[22]	validation_0-merror:0.007115
[23]	validation_0-merror:0.007125
[24]	validation_0-merror:0.006977
[25]	validation_0-merror:0.006945
[26]	validation_0-merror:0.006945
[27]	validation_0-merror:0.00

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1500, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

* XGBClassifier
              (base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1500, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

* 0.9472021192302403

In [ ]:
# a = [100, 200, 300, 400, 600, 800]

# for i in tqdm(a):
#   print(i)
#   forest=RandomForestClassifier(n_estimators=i)
#   forest.fit(x_train_vector, label_train)

#   preds=forest.predict(x_val_vector)
#   f1 = macro_f1(label_val,preds)
#   print(f1)


 17%|█▋        | 1/6 [01:04<05:22, 64.49s/it]

0.8231783924173538



 33%|███▎      | 2/6 [03:12<05:34, 83.65s/it]

0.822552185468849



 50%|█████     | 3/6 [06:31<05:54, 118.05s/it]

...

0.8235748398220635



 83%|████████▎ | 5/6 [17:35<03:52, 232.84s/it]

0.8232041400559027



100%|██████████| 6/6 [26:33<00:00, 265.65s/it]

0.8236638063769065


In [ ]:
x_train_vector

array([[161,   1,   1, ...,   0,   0,   0],
       [  3,  53,   7, ...,   1, 332, 327],
       [ 43,   1,   1, ...,   0,   0,   0],
       ...,
       [  1,   1,   1, ...,   1,   1,   1],
       [  1,  56,   1, ...,  41,  51,  52],
       [ 43,   1,   1, ...,   0,   0,   0]], dtype=int32)

In [ ]:
x_val_vector

array([[126,  26,   1, ...,   1,   1,   1],
       [146,   1,   1, ...,   0,   0,   0],
       [  1,   1,   1, ...,  12,  14,  13],
       ...,
       [  1,   1,   1, ...,  12,  14,  13],
       [ 44,  28,  54, ...,   1,   1,  18],
       [ 43,   1,   1, ...,   0,   0,   0]], dtype=int32)

In [ ]:
# 모델 자체
forest.score(x_val_vector, label_val)

0.9978857233469

In [ ]:
#crosstab으로 확인
pred=forest.predict(x_val_vector)
crosstab = pd.crosstab(label_val, pred, rownames=['real'], colnames=['pred'])
crosstab

pred,0,1,2,3,4,5
real,,,,,,
0,66731,51,0,0,0,4
1,116,26429,0,0,0,2
2,0,0,2,0,0,0
3,6,0,0,795,0,0
4,0,0,0,0,3,0
5,16,2,0,0,0,435
6,3,0,0,0,0,0


In [ ]:
preds=forest.predict(x_val_vector)
probas=forest.predict_proba(x_val_vector)
print(preds.shape)
print(probas.shape)

(94595,)
(94595, 7)


In [ ]:
from collections import Counter
Counter(label_val)

Counter({0: 66786, 1: 26547, 2: 2, 3: 801, 4: 3, 5: 453, 6: 3})

In [ ]:
# 7을 추가
# preds[np.where(np.max(probas, axis=1)<0.9)]=7
# new_crosstab = pd.crosstab(label_val, preds, rownames=['real'], colnames=['pred'])
# new_crosstab

In [ ]:
macro_f1(label_val,preds)

0.8521005548588904

In [ ]:
#crosstab으로 확인
pred=lgb.predict(x_val_vector)
crosstab = pd.crosstab(label_val, pred, rownames=['real'], colnames=['pred'])
crosstab

pred,0,1,2,3,4,5,6
real,,,,,,,
0,66781,30,0,1,0,1,0
1,144,26360,0,0,0,0,0
2,0,0,2,0,0,0,0
3,6,1,0,821,0,0,0
4,0,0,0,0,2,0,0
5,19,3,0,0,0,422,0
6,1,0,0,0,0,0,1


In [ ]:
preds=lgb.predict(x_val_vector)
probas=lgb.predict_proba(x_val_vector)
print(preds.shape)
print(probas.shape)

(94595,)
(94595, 7)


In [ ]:
from collections import Counter
Counter(label_val)

Counter({0: 66813, 1: 26504, 2: 2, 3: 828, 4: 2, 5: 444, 6: 2})

In [ ]:
macro_f1(label_val,preds)

0.9472021192302403

In [ ]:
label_val

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
get_clf_eval(label_val,preds)

오차행렬:
 [[66781    30     0     1     0     1     0]
 [  144 26360     0     0     0     0     0]
 [    0     0     2     0     0     0     0]
 [    6     1     0   821     0     0     0]
 [    0     0     0     0     2     0     0]
 [   19     3     0     0     0   422     0]
 [    1     0     0     0     0     0     1]]

정확도: 0.9978
정밀도: 0.9978
재현율: 0.9978
F1: 0.9978


In [ ]:
# 혼동행렬, 정확도, 정밀도, 재현율, F1, AUC 불러오기
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score

def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    precision = precision_score(y_test, y_pred, average='micro')
    recall = recall_score(y_test, y_pred, average='micro')

    F1 = f1_score(y_test, y_pred, average='micro')

    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))

# 다시 시작

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/로그분석/data/train.csv')
vali = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/로그분석/data/validation_sample.csv')
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/로그분석/data/test.csv')
submission=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/로그분석/data/sample_submission.csv')

In [ ]:
# text 전처리로 문자만 남겨 보았다.
train['full_log'] = train['full_log'].str.replace(r'[0-9,Jan,Feb,Dec,Sep,Oct,Nov,Mar]', '')
test['full_log'] = test['full_log'].str.replace(r'[0-9,Jan,Feb,Dec,Sep,Oct,Nov,Mar]', '')

In [ ]:
train['full_log'].value_counts()

  :: llhs ki: {"y":"""@imsm":"--T::Z""gs":["wig""ss-lli"]"id":"ll":"""":{"mssg":" Liig is""m":"E""sk":"E:  Liig is\     sdRqWihCi (/us/sh/ki/d_mduls/lsish/s/li/s.js::)\     x (/us/sh/ki/d_mduls/lsish/s/li/i_l.js::)\     ss._ikCllk (il/ss/x_ik.js::)"}"mssg":" Liig is"}                                                                                                                                                                                                                                                                                                                                                                                                        89309
  :: llhs ki: {"y":"lg""@imsm":"--T::Z""gs":["""sk_mg"]"id":"mssg":"ild  ll f wk: E:  Liig is"}                                                                                                                                                                                                                                               

In [ ]:
text_train, text_val, label_train, label_val = train_test_split(train['full_log'], train['level'], test_size=0.2, random_state=42, stratify=train['level'])

In [ ]:
import tensorflow as tf

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(text_train)
top_k = len(tokenizer.word_index)

x_train = tokenizer.texts_to_sequences(text_train)
x_val = tokenizer.texts_to_sequences(text_val)

max_length=100

x_train_vector = tf.keras.preprocessing.sequence.pad_sequences(
    x_train, maxlen=max_length, padding='post'
)

x_val_vector = tf.keras.preprocessing.sequence.pad_sequences(
    x_val, maxlen=max_length, padding='post'
)

In [ ]:
x_train_vector

array([[ 11,  12,   1, ...,   0,   0,   0],
       [ 11,  32, 130, ...,   0,   0,   0],
       [ 11,  32,  28, ...,   0,   0,   0],
       ...,
       [ 11,  12,   1, ...,   0,   0,   0],
       [ 11,  12,   1, ...,   0,   0,   0],
       [ 11,  12,   1, ...,   0,   0,   0]], dtype=int32)

* https://statkclee.github.io/model/model-python-xgboost-hyper.html

In [ ]:
# LGBM 모델
import xgboost as xgb ## XGBoost 불러오기
lgb = xgb.XGBClassifier(n_estimators=1000, random_state=42, n_jobs=-1, max_depth= 3)
evals = [(x_val_vector, label_val)]
lgb.fit(x_train_vector, label_train, early_stopping_rounds=50, eval_metric='merror', eval_set=evals, verbose=True)

[0]	validation_0-merror:0.002632
Will train until validation_0-merror hasn't improved in 50 rounds.
[1]	validation_0-merror:0.002611
[2]	validation_0-merror:0.002664
[3]	validation_0-merror:0.002685
[4]	validation_0-merror:0.002675
[5]	validation_0-merror:0.002601
[6]	validation_0-merror:0.002505
[7]	validation_0-merror:0.002474
[8]	validation_0-merror:0.002453
[9]	validation_0-merror:0.002379
[10]	validation_0-merror:0.002379
[11]	validation_0-merror:0.002347
[12]	validation_0-merror:0.002357
[13]	validation_0-merror:0.002357
[14]	validation_0-merror:0.002357
[15]	validation_0-merror:0.002379
[16]	validation_0-merror:0.002379
[17]	validation_0-merror:0.00241
[18]	validation_0-merror:0.0024
[19]	validation_0-merror:0.0024
[20]	validation_0-merror:0.002389
[21]	validation_0-merror:0.0024
[22]	validation_0-merror:0.00241
[23]	validation_0-merror:0.002421
[24]	validation_0-merror:0.002421
[25]	validation_0-merror:0.00241
[26]	validation_0-merror:0.002421
[27]	validation_0-merror:0.002368


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
#crosstab으로 확인
pred=lgb.predict(x_val_vector)
crosstab = pd.crosstab(label_val, pred, rownames=['real'], colnames=['pred'])
crosstab

pred,0,1,2,3,4,5,6
real,,,,,,,
0,66787,23,0,1,0,2,0
1,147,26357,0,0,0,0,0
2,0,0,2,0,0,0,0
3,5,1,0,818,0,4,0
4,0,0,0,0,2,0,0
5,24,1,0,0,0,419,0
6,0,1,0,0,0,0,1


In [ ]:
preds=lgb.predict(x_val_vector)
probas=lgb.predict_proba(x_val_vector)
print(preds.shape)
print(probas.shape)

(94595,)
(94595, 7)


In [ ]:
macro_f1(label_val,preds)

0.9456476510091865

In [ ]:
get_clf_eval(label_val,preds)

오차행렬:
 [[66787    23     0     1     0     2     0]
 [  147 26357     0     0     0     0     0]
 [    0     0     2     0     0     0     0]
 [    5     1     0   818     0     4     0]
 [    0     0     0     0     2     0     0]
 [   24     1     0     0     0   419     0]
 [    0     1     0     0     0     0     1]]

정확도: 0.9978
정밀도: 0.9978
재현율: 0.9978
F1: 0.9978


In [ ]:
#test 데이터 전처리
text_test = test['full_log'].str.replace(r'[0-9,Jan,Feb,Dec,Sep,Oct,Nov,Mar]', '')

x_test = tokenizer.texts_to_sequences(text_test)

x_test_vector = tf.keras.preprocessing.sequence.pad_sequences(
   x_test , maxlen=max_length, padding='post'
)

In [ ]:
results=lgb.predict(x_test_vector)
results_proba=lgb.predict_proba(x_test_vector)
# results[np.where(np.max(results_proba, axis=1) < 0.9)] = 7

In [ ]:
# results=forest.predict(x_test_vector)
# results_proba=forest.predict_proba(x_test_vector)
# # results[np.where(np.max(results_proba, axis=1) < 0.9)] = 7

In [ ]:
submission['level']=results

In [ ]:
submission

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/로그분석/result/submission_lgb2_0512.csv', index=False)

In [ ]:
text_test = test['full_log'].str.replace(r'[0-9]', '<num>')

x_test = tokenizer.texts_to_sequences(text_test)

x_test_vector = tf.keras.preprocessing.sequence.pad_sequences(
   x_test , maxlen=max_length, padding='post'
)

In [ ]:
#CountVectorizer로 벡터화
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer=CountVectorizer(analyzer="word", max_features=10000)

# train_features=vectorizer.fit_transform(train_text)

In [ ]:
# train_features

In [ ]:
# TfidfVectorizer로 백터화

# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer(analyzer="word", max_features=10000) 
# train_features = tfidf_vectorizer.fit_transform(train_text)

In [ ]:
# train_features

> 텍스트 전처리
* https://codetorial.net/tensorflow/natural_language_processing_in_tensorflow_01.html

In [ ]:
# 토큰화랑 시퀸스 변환
import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences


# tokenizer = Tokenizer(num_words = 1000, oov_token="<OOV>")
# tokenizer.fit_on_texts(train_text)
# word_index = tokenizer.word_index

# train_features = tokenizer.texts_to_sequences(train_text)
# train_features = pad_sequences(train_features)

# print(word_index)
# print(train_features)

In [ ]:
# train_features

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(text_train)
top_k = len(tokenizer.word_index)

x_train = tokenizer.texts_to_sequences(text_train)
x_val = tokenizer.texts_to_sequences(text_val)

max_length=100

x_train_vector = tf.keras.preprocessing.sequence.pad_sequences(
    x_train, maxlen=max_length, padding='post'
)

x_val_vector = tf.keras.preprocessing.sequence.pad_sequences(
    x_val, maxlen=max_length, padding='post'
)

# **4. 모델링**

In [ ]:
#훈련 데이터 셋과 검증 데이터 셋으로 분리
TEST_SIZE=0.2
RANDOM_SEED=42

train_x, eval_x, train_y, eval_y=train_test_split(train_features, train_level, test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [ ]:
#랜덤포레스트로 모델링
from sklearn.ensemble import RandomForestClassifier

forest=RandomForestClassifier(n_estimators=10000)

forest.fit(train_x, train_y)

In [ ]:
#모델 검증
forest.score(eval_x, eval_y)

In [ ]:
#crosstab으로 확인
pred=forest.predict(eval_x)
crosstab = pd.crosstab(eval_y, pred, rownames=['real'], colnames=['pred'])
crosstab

+ 새로운 위험요소에 대한 가정 추가
+ => 예측치의 예측 확률이 0.90이하인 경우, 즉 확신이 없을 경우 이상치로 판단 

In [ ]:
preds=forest.predict(eval_x)
probas=forest.predict_proba(eval_x)
print(preds.shape)
print(probas.shape)

In [ ]:
preds[np.where(np.max(probas, axis=1)<0.9)]=7
new_crosstab = pd.crosstab(eval_y, preds, rownames=['real'], colnames=['pred'])
new_crosstab

# **5. 예측**

In [ ]:
#test 데이터 전처리
test_text=list(test['full_log'])
ids=list(test['id'])

In [ ]:
#test 데이터 vectorizer
#주의: fit_transform의 경우 data leakage에 위반될 수 있습니다
# test_features=vectorizer.transform(test_text)

In [ ]:
# 토큰화랑 시퀸스 설정
test_features = tokenizer.texts_to_sequences(test_text)
test_features = pad_sequences(test_features)
print(test_features)

In [ ]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)
results[np.where(np.max(results_proba, axis=1) < 0.9)] = 7

In [ ]:
submission['level']=results

In [ ]:
submission

,id,level
0,1000000,7
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,7
1418913,2418913,1
1418914,2418914,0


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/로그분석/result/submission_0511.csv', index=False)